### ECS 289G Project
Emily Guo
Erjie Zhang

### Preprocess and Tokenization
Here, the multinews dataset as well as the tokenizer are loaded in. The preprocess function will tokenize the documents and summaries.

In [1]:
from transformers import BartTokenizer
from datasets import load_dataset, Dataset

tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
dataset = load_dataset("multi_news")

def preprocess(x):
    tokenized = tokenizer(x["document"], max_length=1024, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(x["summary"], max_length=256, truncation=True, padding='max_length')
    return{
        "input_ids": tokenized["input_ids"], 
        "attention_mask": tokenized["attention_mask"], 
        "labels": labels["input_ids"]
    }

train_dataset = dataset["train"].map(preprocess, batched=True)
dev_dataset = dataset["validation"].map(preprocess, batched=True)
test_dataset = dataset["test"].map(preprocess, batched=True)

c:\Users\EMMIE\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Train

In [2]:
from transformers import BartForConditionalGeneration, Trainer, TrainingArguments

model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')


training_args = TrainingArguments(
    output_dir = "log",
    per_device_train_batch_size = 4,
    per_device_eval_batch_size = 4,
    num_train_epochs = 3,
    weight_decay = 0.01,
    evaluation_strategy = "steps",
    save_strategy = "steps",
    eval_steps = 50,
    # evaluation_strategy = "epoch",
    # save_strategy = "epoch",
    load_best_model_at_end = True,
    push_to_hub = True,
    warmup_steps = 500,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = dev_dataset,
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

: 